In [1]:
## Importamos las librerias necesarias

# Para hacer las requests a la API
import requests

# Para guardar los tokens de acceso de la API
import os

#Para manejar los json que sacamos de la API
import json

import pandas as pd
import csv

# Para parsear a un formato legible los datos de fecha que sacamos de la API
import dateutil.parser


In [2]:
## para poder ver el texto entero en la columna del dataframe
pd.set_option("display.max_colwidth", None)

In [ ]:
## sustituir <bearer_token> por nuestro bearer_token personal y ejecutar la celda
## una vez ejecutada se puede borrar, ya que en la siguiente celda se va a recuperar el token del entorno
## y así ya no se queda visible
# os.environ['TOKEN'] = '<bearer_token>'

In [4]:
## ahora creamos nuestra función auth() que recupera el TOKEN del entorno
def auth():
    return os.getenv('TOKEN')

In [5]:
## creamos una función que coje nuestro bearer token, lo pasa para la autorización y retorna unos HEADERS que usaremos para acceder a la API
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [6]:
## ahora que podemos acceder a la API, construimos la request para el "endpoint" que vamos a usar y los parámetros que queremos pasar
def create_url(keyword, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent"   ## "recent search" baja los tweets de los últimos 7 días
    
    query_params = {'query': keyword,
                'max_results': max_results,
                'expansions': 'attachments.poll_ids,attachments.media_keys,author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
                'media.fields': 'duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics,alt_text',
                'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
                'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                'next_token': {}}
    return (search_url, query_params)

In [7]:
## nos conectamos al endpoint, ahora que tenemos url, headers y los parametros que queremos
## La función mandará la request GET, si todo sale bien (response code 200), retornará la response en formato json
## NOTA: "next_token" está como None por defecto ya que solo queremos saber si existe
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [8]:
# ## creamos la request, vamos a definir los inputs (los que nos interesa sacar de twitter)

bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "samsung lang:es"   ##tweets en español que contengan la palabra "samsung"
max_results = 100   ## según los permisos que tenemos, no se pueden bajar más de 100 a la vez


## ahora creamos la URL y tendremos el response de la API (en formato json). Si todo está correcto saldrá el código 200
url = create_url(keyword, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [9]:
## ahora imprimimos el response en formato json
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "1168319786864054272",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "10026364281",
                        "name": "Apple"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "1174344555111469057",
                        "name": "Samsung Indonesia"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                

### Vamos a quedarnos con los campos que necesitamos incluir en el dataframe final: 
- text  (el texto del tweet)
- username (el autor del tweet)
- created_at (fecha de creación del tweet)
- followers_count (número de seguidores del autor del tweet)
- retweet_count (número de retweets que ha tenido este tweet)
- verified (si el tweet está verificado o no)


In [10]:
## para ver las claves del archivo json que nos hemos bajado
json_response.keys()

dict_keys(['data', 'includes', 'meta'])

In [11]:
json_response["includes"].keys()

dict_keys(['users', 'media', 'tweets', 'places'])

In [12]:
json_response["includes"]["users"][6]

{'url': '',
 'verified': False,
 'profile_image_url': 'https://pbs.twimg.com/profile_images/1437819084352131073/1QWpiExe_normal.jpg',
 'id': '1076383362250690560',
 'protected': False,
 'public_metrics': {'followers_count': 5,
  'following_count': 90,
  'tweet_count': 176,
  'listed_count': 0},
 'location': 'Grasalandia :v',
 'username': 'Oh_GRAN_JUEZ',
 'name': 'EL_JUEZ',
 'created_at': '2018-12-22T07:46:06.000Z',
 'description': '💙💛💙🏆🇦🇷🔟'}

In [13]:
json_response["includes"]["users"][0]["username"]

'rumiantee'

In [14]:
json_response["includes"]["users"][0]["verified"]

False

In [15]:
json_response["includes"]["users"][0]["public_metrics"]["followers_count"]

521

In [16]:
json_response["data"][0].keys()

dict_keys(['conversation_id', 'reply_settings', 'possibly_sensitive', 'author_id', 'id', 'public_metrics', 'context_annotations', 'lang', 'source', 'entities', 'text', 'created_at'])

In [17]:
json_response["data"][0]["created_at"]

'2022-01-19T13:48:03.000Z'

In [18]:
json_response["data"][0]["public_metrics"]["retweet_count"]

0

In [19]:
json_response["data"][0]["text"]

'Oye pero ira a funcionar la ley de tallas? Si mal no estoy, había una norma que prohibía vender celulares sin cargador, y ahí esta Samsung y Apple haciéndolo, igual que la ley que prohibía vender celulares sin radio y también se venden xd'

### Creamos archivos csv para cada una de las 2 keys del archivo json, y así almacenamos los datos
Sucesivamente uniremos los 2 dataframes en uno final

IMPORTANTE: una vez creado los 2 csv, los comentamos, ya que si no a cada vez que ejecutemos el notebook se añadirán las cabeceras de los csv

In [20]:
## aquí salvamos el response en json, pero NO nos sirve para nuestro analisis
## en el paso de la siguiente celda lo guardamos a csv
with open('../data/processed/data_ver.json', 'w') as f:
    json.dump(json_response, f)

In [281]:
# ## creamos el archivo csv para la key "data"  -->part_1

# # csvFile_part_1_ver = open("../data/processed/data_part_1_ver.csv", "a", newline="", encoding='utf-8')
# csvWriter_part_1_ver= csv.writer(csvFile_part_1_ver)

In [282]:
## creamos las cabeceras del archivo csv (vacio de momento) relativas a los datos que quiera guardar
## para la key "data"  -->part_1
# csvWriter_part_1_ver.writerow(['text', 'created_at', 'retweet_count'])
# csvFile_part_1_ver.close()

In [283]:
## creamos el archivo csv para la key "data"  -->part_2
# # csvFile_part_2_ver = open("../data/processed/data_part_2_ver.csv", "a", newline="", encoding='utf-8')
# csvWriter_part_2_ver = csv.writer(csvFile_part_2_ver)

In [284]:
### creamos las cabeceras del archivo csv (vacio de momento) relativas a los datos que quiera guardar
## para la key "data"  -->part_2
# csvWriter_part_2_ver.writerow(['username',  'followers_count', 'verified'])
# csvFile_part_2_ver.close()

### ------------------------------


In [34]:
## creamos una función para añadir los campos necesarios del json al csv 
# cada vez que ejecutemos la celda de bajar tweets, se añadirán a este csv
def append_to_csv_part_1_ver(json_response, fileName):
    ''' 
    Añade los campos necesarios del json al csv
    cada vez se ejecute la celda de bajar tweets.
    Relativo a la clave ["data"] del json
    '''

    counter = 0

    #Abre o crea el target CSV file
    csvFile_part_1_ver = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter_part_1_ver = csv.writer(csvFile_part_1_ver)

    #bucle for para cada tweet relativo a la key "data" del json
    for tweet in json_response['data']:
        
        # 1. Texto del tweet
        text = tweet['text']

        # 3. Fecha y hora en la que se escribió el tweet
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 4. Número de retweets del tweet
        retweet_count = tweet['public_metrics']['retweet_count']
        
        # Unimos todos los datos en una lista
        res = [text, created_at, retweet_count]

        # Añadimos los resultados al CSV
        csvWriter_part_1_ver.writerow(res)
        counter += 1


    # Cuando esté hecho, cerramos el csv
    csvFile_part_1_ver.close()

    # Imprimimos el número de tweets para esta iteración
    print("Número de tweets añadidos en esta request: ", counter) 

In [23]:
## llamamos a la función para incluir las datos en el csv part_1
append_to_csv_part_1_ver(json_response, "../data/processed/data_part_1_ver.csv")

Número de tweets añadidos en esta request:  100


In [ ]:
df_part_1_ver = pd.read_csv("../data/processed/data_part_1_ver.csv")

In [26]:
## creamos una función para añadir los campos necesarios del json al csv 
# cada vez que ejecutemos la celda de bajar tweets, se añadirán a este csv
def append_to_csv_part_2_ver(json_response, fileName):
    ''' 
    Añade los campos necesarios del json al csv
    cada vez se ejecute la celda de bajar tweets.
    Relativo a la clave ["includes"] del json
    '''

    counter = 0

    #Abre o crea el target CSV file
    csvFile_part_2_ver = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter_part_2_ver = csv.writer(csvFile_part_2_ver)

        
    #bucle for para cada tweet relativo a la key "includes" del json
    for tweet in json_response['includes']['users']:
        
        # 2. Username del autor del tweet
        username = tweet['username']
        
        # 5. Número de followers del autor del tweet
        followers_count = tweet['public_metrics']['followers_count']
        
        # 6. Indica si el tweet está verificado o no
        verified = tweet['verified']


        # Unimos todos los datos en una lista
        res = [username, followers_count, verified]
        
        # Añadimos los resultados al CSV
        csvWriter_part_2_ver.writerow(res)
        counter += 1

    # Cuando esté hecho, cerramos el csv
    csvFile_part_2_ver.close()

    # Imprimimos el número de tweets para esta iteración
    print("Número de tweets añadidos en esta request: ", counter) 

In [27]:
## llamamos a la función para incluir las datos en el csv part_2
append_to_csv_part_2_ver(json_response, "../data/processed/data_part_2_ver.csv")

Número de tweets añadidos en esta request:  121


In [ ]:
df_part_2_ver = pd.read_csv("../data/processed/data_part_2_ver.csv")

In [35]:
## creamos una columna temporal para mergear los 2 csv
import numpy as np
df_part_1_ver["temporal"] = np.arange(0, len(df_part_1_ver))
df_part_2_ver["temporal"] = np.arange(0, len(df_part_2_ver))

In [ ]:
df_final_ver = pd.merge(df_part_1_ver,df_part_2_ver, how = "inner",  on=["temporal"])
df_final_ver = df_final_ver.drop("temporal", axis = 1)


In [ ]:
## aplicamos el drop duplicates por si hubiera filas duplicadas
df_final_ver.drop_duplicates()

In [294]:
## guardo el dataframe final en un csv 
## lo guardo en la ruta de la carpeta con la API que vamos a crear para el proyecto
df_final_ver.to_csv("../API_twitter/tweets_ver.csv", index=False) 

In [37]:
## lo abro para comprobar que esté bien
df_ver = pd.read_csv("../API_twitter/tweets_ver.csv")
df_ver

,text,created_at,retweet_count,username,followers_count,verified
0,RT @LuisGyG: 70 ciudadanos de Coreal del Sur (incluidos algunos representantes de Samsung y la fabricante de chips SK Hynix) dieron positiv…,2022-01-14 14:24:59+00:00,1,JoiaribElnatan,16056,False
1,"@anabel0944 Xa a mi gusto el iphone pa grabar videos en 4k, pero me gusta mas para fotos el s21 ultra ahora ese celular samsung puedes grabar 4k y 8k🥺😍",2022-01-14 14:24:35+00:00,0,LuisGyG,64264,True
2,"RT @hyyunmei: K-Stay: las típicas primas milipilis que tiene todo iPhone, Gucci, todo re caro y le pasas un samsung o motorola y empieza a…",2022-01-14 14:23:47+00:00,2,GiXnoxD,416,False
3,"70 ciudadanos de Coreal del Sur (incluidos algunos representantes de Samsung y la fabricante de chips SK Hynix) dieron positivo a #COVID tras asistir al #CES2022 en Las Vegas, según @Reuters. La CTA, organizadora del CES, no pudo confirmar el número de positivos en el evento. https://t.co/IfbAhsgesN",2022-01-14 14:22:48+00:00,1,anabel0944,180,False
4,"@meinOddity si tiene Samsung te sale una app llamada game launcher, ahí deben de estar todos los juegos descargados que tengas, abajo del icon del juego aparace una opción llamada ""detalles"", y en esa te dice cuantas horas vas jugando cada juego",2022-01-14 14:22:46+00:00,0,MarSt4y,16,False
...,...,...,...,...,...,...
95,RT @SaraOdemiOdeck: Es poco probable que lo termine a si que lo comparto asi 😆 Mi primera ilustracion hecha completamente en mi samsung tab…,2022-01-14 12:36:14+00:00,3,csnwz_,3248,False
96,"@xJulio89x @ParamountPlusLA Pues yo tengo Samsung y no me aparece la aplicación, es de modelo reciente",2022-01-14 12:34:00+00:00,0,m__casti,3647,False
97,Samsung anunció nuevas funciones y tecnología para los hogares en CES 2022 @SamsungArg https://t.co/F3mZlOWvrR,2022-01-14 12:33:10+00:00,0,botxboxseriesx,1568,False
98,Hoy lo lograré.\n\nLo juro.\n\nLo que sea por twittear desde un SAMSUNG SmartDildo™ https://t.co/BmwYTFhrR3,2022-01-14 12:33:01+00:00,0,chalioDMX,79,False
